In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [24]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.973254683870284, 8.74478833984142, 8.52834654020913]


# Flipchip

In [16]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [17]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [18]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [19]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [20]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [21]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [13]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 11:15AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:15AM [load_ansys_project]: 	Opened Ansys App
INFO 11:15AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:15AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project25
INFO 11:15AM [connect_design]: No active design found (or error getting active design).
INFO 11:15AM [connect]: 	 Connected to project "Project25". No design detected


True

In [22]:
hfss.activate_ansys_design("flip20_res_coup", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

11:19AM 27s WARNING [activate_ansys_design]: The design_name=flip20_res_coup was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 11:19AM [connect_design]: 	Opened active design
	Design:    flip20_res_coup [Solution type: Eigenmode]
WARNING 11:19AM [connect_setup]: 	No design setup detected.
WARNING 11:19AM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:19AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [21]:
hfss.close()

NameError: name 'hfss' is not defined

# LOM Analysis

In [22]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [23]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.49',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.49',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [25]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 11:27PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0am9fdnd.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5490000000, Maxwell, 1, False
INFO 11:27PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp9ijvi1v0.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5490000000, Maxwell, 1, False
INFO 11:27PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp1ry6z8bo.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5490000000, Maxwell, 2, False
INFO 11:27PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmplcimro6f.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5490000000, Maxwell, 3, False
INFO 11:27PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpg3lxir9e.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5490000000, Maxwell, 4, False
INFO 11:27

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,291.470,-73.13100,-70.7980,-80.83900
pad_bot_Q0,-73.131,105.99000,-32.0920,-0.16105
pad_top_Q0,-70.798,-32.09200,105.7400,-2.15620
quad_coupler_c0,-80.839,-0.16105,-2.1562,83.49600


In [26]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 5.99
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.419612 [GHz]
EC 274.158597 [MHz]
EJ 14.854145 [GHz]
alpha -312.742707 [MHz]
dispersion 5.293163 [KHz]
Lq 10.995564 [nH]
Cq 70.653367 [fF]
T1 624.274788 [us]

**Coupling Properties**

tCqbus1 -1.000252 [fF]
gbus1_in_MHz -12.386594 [MHz]
χ_bus1 -0.191271 [MHz]
1/T1bus1 254.943730 [Hz]
T1bus1 624.274788 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.655493,300.119251,14.854145,-345.009143,12.888569,[-52.685736103712884],[-8.440936264931429],8.440936,52.685736
2,5.685177,303.480801,14.854145,-349.218944,14.339471,[-25.28956895237742],[-2.2439400745562135],2.243940,25.289569
3,5.587028,292.44706,14.854145,-335.428276,10.033276,[-19.64141527304513],[-0.8717683442970101],0.871768,19.641415
4,5.574523,291.057877,14.854145,-333.69755,9.578196,[-13.332809385348677],[-0.3820642273427684],0.382064,13.332809
5,5.522541,285.323349,14.854145,-326.566273,7.879348,[-12.003902383975744],[-0.2542834057903478],0.254283,12.003902
6,5.4643,278.975011,14.854145,-318.696316,6.3029,[-11.956418825250873],[-0.2059840998423562],0.205984,11.956419
7,5.435507,275.866301,14.854145,-314.851913,5.634175,[-12.2691975995339],[-0.19739048227100658],0.197390,12.269198
8,5.419612,274.158597,14.854145,-312.742707,5.293163,[-12.386594098471784],[-0.19127135406641982],0.191271,12.386594


In [27]:
c1.sim.close()

Warning! 6 COM references still alive
Ansys will likely refuse to shut down
